In [29]:
from pymongo import MongoClient
from pprint import pprint
import json

client = MongoClient("localhost", 27017)
client.drop_database("userData")

db = client.userData
with open("array-data.json") as f:
    file_data = json.load(f)
db.friends.insert_many(file_data)


In [30]:
for x in db.friends.aggregate(
    [
        {"$unwind": "$examScores"},
        {"$project": {"_id": 1, "name": 1, "score": "$examScores.score"}},
        {"$sort": {"examScores": -1}},
        {
            "$group": {
                "_id": "$_id",
                "name": {"$first": "$name"},
                "max_score": {"$max": "$score"},
            }
        },
        {"$sort": {"max_score": -1}},
    ]
):
    pprint(x)


{'_id': ObjectId('64570702d8954984f8e038f8'), 'max_score': 88.5, 'name': 'Max'}
{'_id': ObjectId('64570702d8954984f8e038fa'),
 'max_score': 75.1,
 'name': 'Maria'}
{'_id': ObjectId('64570702d8954984f8e038f9'), 'max_score': 74.3, 'name': 'Manu'}


In [31]:
client.close()
